In [5]:
import ee
import os
import json
from datetime import datetime

Initialize Earth Engine

In [6]:
ee.Authenticate()
ee.Initialize()

Define Fire Events

In [7]:
fires = {
    "Dixie": ("2021-07-13", "2021-10-25", [ -121.3, 40.0 ]),
    "Camp": ("2018-11-08", "2018-11-25", [ -121.4, 39.8 ]),
    "Bootleg": ("2021-07-01", "2021-08-31", [ -121.2, 42.6 ]),
    "Caldor": ("2021-08-01", "2021-10-01", [ -120.6, 38.7 ]),
    "Troublesome": ("2020-10-01", "2020-11-15", [ -106.5, 40.2 ]),
    "Carr": ("2018-07-23", "2018-09-04", [ -122.6, 40.7 ]),
    "Thomas": ("2017-12-04", "2018-01-12", [ -119.1, 34.5 ]),
    "Creek": ("2020-09-04", "2020-12-24", [ -119.3, 37.3 ]),
    "Woolsey": ("2018-11-08", "2018-11-21", [ -118.8, 34.2 ]),
    "Glass": ("2020-09-27", "2020-10-20", [ -122.4, 38.6 ]),
}

output_dir = "../data/raw/spread"
os.makedirs(output_dir, exist_ok=True)

Helper Functions

In [8]:
def export_image_to_local(image, region, scale, out_path):
    """Download EE image to GeoTIFF using getDownloadURL() approach."""
    url = image.clip(region).getDownloadURL({
        'scale': scale,
        'region': region,
        'format': 'GEO_TIFF'
    })
    import requests
    r = requests.get(url)
    with open(out_path, 'wb') as f:
        f.write(r.content)
    print(f"Saved: {out_path}")
    
def get_burned_area_image_and_aoi(aoi, start, end):
    buffered_aoi = aoi.buffer(5000)
    modis = ee.ImageCollection("MODIS/006/MCD64A1") \
        .filterDate(start, end) \
        .filterBounds(buffered_aoi) \
        .select("BurnDate")
    
    modis_mean = modis.mean()
    
    if modis_mean.bandNames().size().getInfo() == 0:
        print("[Fallback] MODIS empty — using VIIRS instead.")
        viirs = ee.ImageCollection("NOAA/VIIRS/001/VNP14A1") \
            .filterDate(start, end) \
            .filterBounds(buffered_aoi) \
            .select("MaxFRP")
        viirs_mean = viirs.mean()
        if viirs_mean.bandNames().size().getInfo() == 0:
            print("[Warning] Both MODIS and VIIRS empty. Using buffered AOI only.")
            return None, buffered_aoi
        else:
            return viirs_mean.gt(0), buffered_aoi
    else:
        return modis_mean.gt(0), modis_mean.gt(0).geometry()

Fire Event Loop

In [9]:
for fire_name, (start_date, end_date, coords) in fires.items():
    print(f"\nProcessing fire: {fire_name}")
    fire_dir = os.path.join(output_dir, fire_name)
    os.makedirs(fire_dir, exist_ok=True)

    # Try MODIS burned area geometry first
    try:
        burned = ee.ImageCollection("MODIS/061/MCD64A1") \
            .filterDate(start_date, end_date) \
            .select("BurnDate") \
            .mean()

        aoi = burned.geometry()
        if aoi.isUnbounded() or aoi.area().getInfo() == 0:
            raise Exception("Invalid MODIS AOI")

        print(f"AOI for {fire_name}: Using MODIS burned area footprint")
    except Exception as e:
        print(f"[Warning] MODIS AOI failed for {fire_name}, using fallback buffer. ({e})")
        aoi = ee.Geometry.Point(coords).buffer(20000)  # 20 km fallback

    # Save AOI as GeoJSON for inspection
    try:
        aoi_geojson = aoi.getInfo()
        with open(os.path.join(fire_dir, "aoi.geojson"), "w") as f:
            json.dump(aoi_geojson, f)
    except Exception as e:
        print(f"[Warning] Could not save AOI GeoJSON for {fire_name}: {e}")

    # Skip export if AOI invalid
    if aoi.area().getInfo() == 0:
        print(f"[Skipped] {fire_name} AOI area is zero — no export performed.")
        continue

    # Define core datasets
    burned_area = burned
    ndvi_prefire = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterDate(ee.Date(start_date).advance(-90, "day"), start_date) \
        .median().normalizedDifference(["B8", "B4"]).rename("NDVI")

    landcover = ee.ImageCollection("ESA/WorldCover/v200").first().select("Map")

    temp_mean = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
        .filterDate(start_date, end_date) \
        .select("temperature_2m").mean()

    precip_sum = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
        .filterDate(start_date, end_date) \
        .select("precipitation").sum()

    elevation = ee.Image("USGS/SRTMGL1_003").select("elevation")

    # Export locally (scaled to 100m)
    scale = 100
    try:
        export_image_to_local(burned_area, aoi, scale, os.path.join(fire_dir, "burned_area.tif"))
        export_image_to_local(ndvi_prefire, aoi, scale, os.path.join(fire_dir, "ndvi_prefire.tif"))
        export_image_to_local(landcover, aoi, scale, os.path.join(fire_dir, "landcover.tif"))
        export_image_to_local(temp_mean, aoi, scale, os.path.join(fire_dir, "temperature_mean.tif"))
        export_image_to_local(precip_sum, aoi, scale, os.path.join(fire_dir, "precipitation_sum.tif"))
        export_image_to_local(elevation, aoi, scale, os.path.join(fire_dir, "elevation.tif"))
    except Exception as e:
        print(f"[Error] Export failed for {fire_name}: {e}")

print("\nDone. Saved under data/raw/spread/<fire_name>/")


Processing fire: Dixie
[Warning] MODIS AOI failed for Dixie, using fallback buffer. (Invalid MODIS AOI)
Saved: ../data/raw/spread\Dixie\burned_area.tif
Saved: ../data/raw/spread\Dixie\ndvi_prefire.tif
Saved: ../data/raw/spread\Dixie\landcover.tif
Saved: ../data/raw/spread\Dixie\temperature_mean.tif
Saved: ../data/raw/spread\Dixie\precipitation_sum.tif
Saved: ../data/raw/spread\Dixie\elevation.tif

Processing fire: Camp
[Warning] MODIS AOI failed for Camp, using fallback buffer. (Invalid MODIS AOI)
[Error] Export failed for Camp: Expression evaluates to an image with no bands.

Processing fire: Bootleg
[Warning] MODIS AOI failed for Bootleg, using fallback buffer. (Invalid MODIS AOI)
Saved: ../data/raw/spread\Bootleg\burned_area.tif
Saved: ../data/raw/spread\Bootleg\ndvi_prefire.tif
Saved: ../data/raw/spread\Bootleg\landcover.tif
Saved: ../data/raw/spread\Bootleg\temperature_mean.tif
Saved: ../data/raw/spread\Bootleg\precipitation_sum.tif
Saved: ../data/raw/spread\Bootleg\elevation.tif